In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from konlpy.tag import Twitter


import pandas as pd
import numpy as np

import pickle

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Load Dataset

- 0: 정치
- 1: 경제
- 2: 사회
- 3: 생활/문화
- 4: 세계
- 5: IT

In [5]:
article_df = pd.read_pickle("article_2016-06-01.plk")
print(len(article_df))
article_df.tail(3)

2327


,newsid,oid,newspaper,title,link,comment,likeit,content,date,category
2324,3179706,23,조선일보,"[어수웅의 르네상스人] 종이 사전을 삼킨 남자, &#39;웹 사전&#39;을 낳다",http://news.naver.com/main/read.nhn?mode=LSD&m...,0,2,"[정철 카카오 웹 사전 기획자] 책 '검색, 사전을 삼키다' 펴내… 연세대에서 사전...",2016-06-01,5
2325,2976797,20,동아일보,"中, M&A로 특허 포식… 한국 미래산업 삼킨다",http://news.naver.com/main/read.nhn?mode=LSD&m...,13,9,"[동아일보]中, 로봇-IoT-바이오 등 M&A; 규모… 올해 5월에 이미 작년 기록...",2016-06-01,5
2326,2620983,25,중앙일보,구글·애플 대항마…토종 앱 장터 원스토어 떴다,http://news.naver.com/main/read.nhn?mode=LSD&m...,123,9,"이통 3사, 네이버 손잡고 시장 40% 점유 목표| 양강 구도에 도전장…앱 생태계 ...",2016-06-01,5


### Split Test Train

In [6]:
X_train, X_test, y_train, y_test = train_test_split(article_df.content, article_df.category, test_size = 0.1, random_state=1)

# randomstate = 1 ---> 셔플해서 섞은 다음에 split할 수 있도록 

In [8]:
X_train.shape[0], y_test.shape[0]

(2094, 233)

In [9]:
y_train.shape[0], y_test.shape[0]

(2094, 233)

### Make Model

In [12]:
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model = clf.fit(X_train, y_train)

pipeline의 이유 : 

In [13]:
model

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...rue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

### confusion matrix & classification report

In [14]:
from sklearn.metrics import confusion_matrix, classification_report

In [18]:
y_pred = model.predict(X_test)

In [19]:
y_pred

array([1, 2, 2, 2, 1, 0, 0, 1, 2, 2, 2, 1, 1, 0, 2, 0, 0, 2, 0, 2, 2, 2,
       1, 2, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 0, 2, 2, 1, 2, 0,
       0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 1, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0,
       0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 2, 2, 0, 1, 2, 1, 2,
       2, 2, 1, 1, 2, 0, 1, 2, 2, 1, 0, 2, 1, 1, 2, 2, 0, 1, 2, 2, 0, 1,
       2, 2, 0, 0, 1, 1, 2, 1, 2, 2, 2, 0, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 0, 1, 2, 2, 1, 2, 0, 2, 1, 2, 2, 0, 2, 0, 0, 0, 0, 2, 1,
       0, 2, 1, 2, 0, 2, 2, 2, 1, 2, 1, 0, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0,
       1, 0, 2, 2, 1, 0, 2, 2, 2, 1, 2, 2, 1, 2, 0, 0, 2, 1, 2, 2, 2, 2,
       0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2,
       1, 0, 2, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2])

In [22]:
print(confusion_matrix(y_test, y_pred))

[[42  0  4  0  0  0]
 [ 4 32  5  0  0  0]
 [ 2  2 79  0  0  0]
 [ 0  0 21  0  0  0]
 [15  3 10  0  1  0]
 [ 0  7  6  0  0  0]]


In [23]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.67      0.91      0.77        46
          1       0.73      0.78      0.75        41
          2       0.63      0.95      0.76        83
          3       0.00      0.00      0.00        21
          4       1.00      0.03      0.07        29
          5       0.00      0.00      0.00        13

avg / total       0.61      0.66      0.56       233



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [28]:
# 알파값 조정

clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha = 0.01))
])

model = clf.fit(X_train, y_train)

In [29]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[41  0  3  1  1  0]
 [ 1 33  0  5  1  1]
 [ 1  2 77  2  1  0]
 [ 0  0  2 18  0  1]
 [ 1  1  2  0 25  0]
 [ 0  1  1  0  0 11]]
             precision    recall  f1-score   support

          0       0.93      0.89      0.91        46
          1       0.89      0.80      0.85        41
          2       0.91      0.93      0.92        83
          3       0.69      0.86      0.77        21
          4       0.89      0.86      0.88        29
          5       0.85      0.85      0.85        13

avg / total       0.88      0.88      0.88       233



### Prediction

In [32]:
X_test = X_test.reset_index(drop=True)
X_test[0]

'인천 송도 삼성바이오로직스 현장을 가다“삼성이 처음 핸드폰 사업을 시작할 때 망치로 핸드폰을 깨부수는 등 온갖 실험을 했던 동영상도 보여주며 ‘이렇게 시작해 휴대폰 사업도 성공했다’고 어필하기도 했습니다.”지난달 31일 삼성그룹의 신수종 사업인 바이오제약 분야의 대표 회사인 삼성바이오로직스를 찾은 기자에게 윤호열 상무는 회사의 지난 5년을 이렇게 회고했다.“처음에는 거래선들이 ‘과연 너희들이 잘할 수 있겠느냐’고 의구심을 떨치지 못하더군요. 아무리 삼성이 글로벌 기업이지만 바이오분야는 처음이기 때문이었죠.”삼성바이오로직스의 5년은 역량을 키운 기간이자 글로벌 제약사들에게 인정받기 위한 ‘분투의 시간’이었다. 삼성바이오로직스가 바이오의약품 생산 세계 3위가 된 건 우연이 아니었다.자동차를 타고 뿌연 미세먼지를 헤치고 인천 연수구 송도동 바이오대로를 달리자 삼성바이오로직스의 웅장한 건물들이 나타났다. 맞은편에선 제3공장 건설이 한창이었다. 도로명은 처음 ‘지식대로’였다가 지난 4월 ‘바이오대로’로 바뀌었다. 삼성바이오로직스를 비롯해 셀트리온, 동아쏘시오홀딩스의 바이오의약품공장 등 25개 바이오기업이 입주해서다.관계자들의 안내로 제2공장 홍보관을 거쳐 내부로 들어서자 1만5000L급 세포배양 탱크 10기가 내려다보였다. 무균복을 입은 직원들이 생산 시설의 이곳저곳을 꼼꼼히 살피고 있었다‘송도캠퍼스’로 불리는 공장시설은 제1공장(생산능력 3만L)과 2공장(〃 15만L), 지난해 11월 공사를 시작한 제3공장(〃 18만L·착공), R&D센터 등으로 이뤄졌다. 주로 중형 바이오의약품을 생산하는 제1공장은 2013년 7월 가동을 시작했고 지난해 11월 미국 식품의약국(FDA)으로부터 제조승인을 획득, 북미시장의 길을 열었다. 지난 2월 시가동을 시작한 제2공장은 주로 대형 상업 바이오의약품을 생산하며, 생산능력은 15만L로 단일 공장으론 세계 최대 규모이다.삼성바이오로직스는 신약 개발이 아닌 첨단 바이오의약품 제조가 중심인 회사다. 즉 최고의 바이오의약품 제조 프랙티스를

In [35]:
result = model.predict([X_test[0], X_test[1], X_test[2]])
result

array([1, 2, 0])

In [36]:
classifiation_dict = {
    0: "정치",
    1: "경제",
    2: "사회",
    3: "생활/문화",
    4: "세계",
    5: "IT",
}

In [39]:
for idx, category in enumerate(result):
    print(classifiation_dict[category], X_test[idx][:50])

경제 인천 송도 삼성바이오로직스 현장을 가다“삼성이 처음 핸드폰 사업을 시작할 때 망치로 핸드폰
사회 [구의역 사고를 보며] 청년을 위한 나라는 없다 [ 주동식 지역평등시민연대 대표] 요즘 사
정치  [한겨레] 구의역·남양주 사고에 입법대책 분주더민주, 직접 고용법 제정안 발의국민의당, 


### Save & Load Model

In [40]:
pickle.dump(model, open("classification.plk", "wb"))

In [41]:
load_model = pickle.load(open("classification.plk", "rb"))

In [44]:
test_str1 = "데이터 사이언스는 어렵고 돈이 많이 들지만 4차 산업혁명에서 굉장한 순이익을 가져올 수 있다."
test_str2 = "방탄소년단이 가지는 경제적 가치는 매우 커"

In [53]:
classifiation_dict[load_model.predict([test_str1])[0]], \
classifiation_dict[load_model.predict([test_str2])[0]]

('경제', '생활/문화')

In [49]:
classifiation_dict[load_model.predict([test_str2])[0]]

'생활/문화'

In [52]:
load_model.predict_proba([test_str1])

array([0.20236396, 0.38392412, 0.26241059, 0.03323916, 0.03615464,
       0.08190752])